In [2]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
import math
import models.self_model as models
import get_data
import models.resnext as resnext

In [4]:
data_path = "C://Users//shjdl//Desktop//DM_project//data//"
test_path = data_path+"test.csv"

Resnet50_path = "./model-Resnext29-[all]-2019.04.15.pt"
model = resnext.resnext29_8_64()
model = torch.load(Resnet50_path)

model.eval()

test_raw_data = pd.read_csv(test_path, iterator=True)
test_data = test_raw_data.get_chunk(10000).values.astype('int')

C:\Users\shjdl\Desktop\DM_project\DM_Kaggle_lza\models\resnext.py:79: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(m.weight)
D:\tf\envs\tfgpu\lib\site-packages\torch\serialization.py:425: SourceChangeWarning: source code of class 'models.resnext.CifarResNeXt' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [5]:
import get_data
simple_transform = transforms.Compose([
                    transforms.ToTensor(),
#                     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                    ])
test_dataset = get_data.TestDataset(test_data,10000,simple_transform)

In [5]:
print(test_data[0])

[  0 158 159 ... 124 129 110]


In [6]:
import csv
datetime =time.strftime('%Y.%m.%d',time.localtime(time.time()))
result_path = data_path+"result-" +datetime+ ".csv"

test_loader = data.DataLoader(test_dataset, batch_size=1, shuffle=False, drop_last=False)
out = open(result_path,'a', newline='')
csv_write = csv.writer(out)
csv_write.writerow(["Id","Category"])

13

In [7]:
result = []
for image,imageid in tqdm(test_loader,leave=False,disable=False):
    image = image.type(torch.FloatTensor)
    if torch.cuda.is_available():
        image = image.cuda()
        model = model.cuda()
    out = model(image)
    pred = torch.max(out, 1)[1]
    result.append([int(imageid) , pred.item()])

In [8]:
print(len(result))

10000


In [9]:
for i in range(0,10000):
    csv_write.writerow(result[i])

In [10]:
for i in range(9361,9800):
    csv_write.writerow(result[i])